In [7]:
import numpy as np
import matplotlib.pyplot as plt
import itertools

In [8]:
def get_result(time_window):
    spike_count = list()
    predict_max = -1*np.ones(num_test)
    first_spikes = dur_test*np.ones((num_output,num_test))
    fastest = -1*np.ones(num_test)
    latency = dur_test*np.ones(num_test)
    correct_latency = np.zeros(num_test)
    digit_latency = dur_test*np.ones((num_digit,num_test))
    for i in range(num_output):
        index_i = np.where(spikes[:,0] == i)
        spike_train = spikes[index_i, 1][0]
        
        for key, igroup in itertools.groupby(spike_train, lambda x: x // (dur_test+silence)):
            test_id = int(key)
            first_spikes[i][test_id] = list(igroup)[0] - test_id*(dur_test+silence)
        
        ind = np.where(np.mod(spike_train ,(dur_test+silence)) <= time_window)[0]
        temp = np.histogram(spike_train[ind], bins=range(0, (dur_test+silence)*num_test+1,dur_test+silence))[0]
        spike_count.append(temp)
        
    spike_count = np.array(spike_count)
    for i in range(num_test):
        if max(spike_count[:,i]) > 0:
            label = np.argmax(spike_count[:,i])//num_cluster
            predict_max[i] = label
            correct_latency[i] = first_spikes[np.argmax(spike_count[:,i])][i]
        fastest[i] = np.argmin(first_spikes[:,i])//num_cluster
        a = np.reshape(first_spikes[:,i], (num_digit, num_cluster))
        digit_latency[:, i] = a.min(axis=1)
        
    latency = np.min(first_spikes, axis=0)
    return predict_max, fastest, latency, correct_latency, digit_latency

In [9]:
def get_latency():
    respond = 1000*np.ones(num_test)
    for i in range(num_test):
        if len(spikes) > 0:
            resp_train = np.where(spikes[:,1]>i*(dur_test+silence))[0]
            if len(resp_train) > 0:
                resp_ind = resp_train[0]
                latency = spikes[resp_ind, 1] - i*(dur_test+silence)
                respond[i] = latency
    return respond

In [10]:
# import mnist_utils as pois
# test_x, test_y = pois.get_test_data()

In [11]:
import scipy.io as sio
tmp_y = sio.loadmat('mnist.mat')['test_y']
tmp_y = np.argmax(tmp_y, axis=0)

In [12]:
num_digit = 10
num_cluster = 1
num_output = num_cluster*num_digit
dur_test = 1000
silence = 200
num_test = 100
num_mnist = 10000

predict_max = -1*np.ones(num_mnist)
fastest_neuron = -1*np.ones(num_mnist)
respond_time = dur_test*np.ones(num_mnist)
correct_latency = dur_test*np.ones(num_mnist)
digit_latency = dur_test*np.ones((num_digit,num_mnist))

result_timew = []
# time_ws = range(10,100,10)
# time_ws.extend(range(100,1001,100))
time_ws =[1000]

for time_w in time_ws:
    for test_offset in range(0, num_mnist, num_test):
        spike_f = 'spikes/spike_scaled_%d.npy'%(test_offset)
        spikes = np.load(spike_f)
    #     plt.plot([i[1] for i in spikes], [i[0] for i in spikes], ".", )
        plt.show()
        predict, fastest, latency, correct_l, digit_l = get_result(time_w)
        predict_max[test_offset:test_offset+num_test] = predict
        fastest_neuron[test_offset:test_offset+num_test] = fastest
        respond_time[test_offset:test_offset+num_test] = latency
        correct_latency[test_offset:test_offset+num_test] = correct_l
        digit_latency[:,test_offset:test_offset+num_test] = digit_l
#         print sum(predict == tmp_y[test_offset:test_offset+num_test])
    result_timew.append(sum(predict_max == tmp_y))
print result_timew

[9702]


In [123]:
a = result_timew
np.save('result_timewindow_relu.npy', a)
print time_ws

[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]


In [53]:
print respond_time.mean()-0.96, respond_time.std()

31.4522 3.82918936069


In [54]:
correct = predict_max == test_y
input_l = np.zeros(num_digit)
ca = np.zeros(num_digit)
for i in range(num_digit):
    ind = np.where(test_y==i)[0]
    input_l[i] = respond_time[ind].mean()-0.96
    ca[i] = correct[ind].sum()*100./ind.shape[0]
print input_l.mean(), input_l.std(), ca.mean(), ca.std()

31.4301308081 0.767175477304 97.0054518454 1.71051216427


In [71]:
spike_f = 'spikes/spike_scaled_%d.npy'%(0)
spikes = np.load(spike_f)

In [82]:
time_window = 1200
index_i = np.where(spikes[:,0] == 0)
spike_train = spikes[index_i, 1][0]
temp = np.where(np.mod(spike_train ,(dur_test+silence)) <= time_window)[0]
print len(temp),len(spike_train)

3184 3184
